In [79]:
#  Седьмая (и последняя) лаба будет посвящена нейронным сетям. 
#     Для этого нужно взять датасет MNIST и сделать однослойный\многослойный перцептрон для распознавания цифр 0-9. 
#     При этом обязательно, чтобы можно было посмотреть на каких цифрах ваш классификатор ошибается, 
#     а так же должна быть возможность ручного ввода изображений для распознавания и дообучения на них. 
#     Нужно грамотно уметь объяснять все про backpropagation и функции активации. Датасет и инструкция, 
#     как из него извлекать данные лежат здесь: http://yann.lecun.com/exdb/mnist/index.html 
# метод обратного распространения ошибки
# Описание алгоритма: https://habrahabr.ru/post/198268/, 
# http://cyberleninka.ru/article/n/primenenie-svertochnoy-neyronnoy-seti-dlya-raspoznavaniya-rukopisnyh-tsifr
# http://mechanoid.kiev.ua/ml-lenet.html

In [160]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15    

import pandas as pd
import random as rd
import pylab as pl
import numpy as np
from tkinter import *
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
import os, struct
from math import exp
from array import array as pyarray
from sklearn.cross_validation import KFold
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score, f1_score
from scipy.spatial.distance import euclidean, cityblock, cosine, correlation
from IPython.display import display, HTML
from sklearn.metrics import mean_squared_error
plt.rcParams["figure.figsize"]=[15,10]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 15 seconds


In [118]:
from mnist import MNIST

mndata = MNIST('./samples')

# size 28 * 28, train count = 60000, test count = 10000
image_size = 28 # n = 28 ** 2

train_images, train_labels = mndata.load_training()
train_size = len(train_images)

# random.shuffle(trains_images) ???

test_images, test_labels = mndata.load_testing()
test_size = len(test_images)

n = 25

train_pca = np.array(train_images + test_images)
pca = PCA(n_components=n).fit(train_pca)


In [123]:
# init 


def sumprod(a, b):
    return sum(map(lambda x: x[0] * x[1], zip(a, b)))

m = 10 # количество выходных y нейронов
p = 50 # TODO количество скрытых z нейронов


v = [[rd.random() - 0.5 for j in range(p)] for i in range(n)] # Смещение скрытого нейрона j
v0 = [rd.random() - 0.5 for j in range(p)]
w = [[rd.random() - 0.5 for k in range(m)] for j in range(p)] # Смещение нейрона на выходе
w0 = [rd.random() - 0.5 for k in range(m)]

alpha = 0.5 # Скорость обучения

In [124]:
# функция активации (competitive function)

f = lambda x: 1 / (1 + exp(-x)) # [0..1]
f1 = lambda x: f(x) * (1 - f(x)) # z1'(x)

In [136]:
# neural network

def nn_go(x, ans):
    global v, w, v0, w0
    t = [0] * m
    t[ans] = 1
    xx = pca.transform([x])
    xx = preprocessing.normalize(xx)
    xx = xx[0]    
    # step 4. evaluate z
    z_in = [0] * p # Суммарное значение подаваемое на вход скрытого нейрона
    for j in range(p):
        z_in[j] = sumprod([v[i][j] for i in range(n)], xx) + v0[j]
    zz = list(map(f, z_in)) # Скрытый нейрон
    
    # step 5. evaluate y
    y_in = [0] * m # Суммарное значение подаваемое на вход скрытого нейрона
    for k in range(m):
        y_in[k] = sumprod([w[j][k] for j in range(p)], zz) + w0[k]
    yy = list(map(f, y_in)) # Нейрон на выходе    
    
    if (ans == -1):
        res = max(yy)
        for k in range(m):
            if yy[k] == res:
                return k
    
    # step 6. evaluate y error
    sigmak = [(t[k] - yy[k]) * (f1(y_in[k])) for k in range(m)]
    delta_w = [[alpha * sigmak[k] * zz[j] for k in range(m)] for j in range(p)]
    delta_w0 = [alpha * sigmak[k] for k in range(m)]
    
    # step 7. evaluate z error
    sigma_in = [sumprod(sigmak, w[j]) for j in range(p)]
    sigmaj = [sigma_in[j] * f1(z_in[j]) for j in range(p)]
    delta_v = [([alpha * sigmaj[j] * xx[i] for j in range(p)]) for i in range(n)] # TODO
    delta_v0 = [alpha * sigmaj[j] for j in range(p)]
    
    # change weights
    v = [[v[i][j] + delta_v[i][j] for j in range(p)] for i in range(n)] # Смещение скрытого нейрона j
    v0 = [v0[j] + delta_v0[j] for j in range(p)]
    w = [[w[j][k] + delta_w[j][k] for k in range(m)] for j in range(p)] # Смещение нейрона на выходе
    w0 = [w0[k] + delta_w0[k] for k in range(m)]
    
    return mean_squared_error(yy, t)

In [157]:
# neural learning
for steps in range(2):
    mse = 0.0
    for num in range(train_size): #train_size
        mse += nn_go(train_images[num], train_labels[num])    
        if (num % 1000 == 0):
            print(num, end=' ')
    print(mse / train_size)

# 1. mse = 0.0112276998947
# 2. mse = 0.00955731831638
# 3. mse = 0.00871734712824
# 5. mse = 0.00769748620968

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 0.00808093830118
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 0.00769748620968


In [159]:
def getNumber(x):
    res = nn_go(x, -1)    
    return res

# neural test
ok = [0] * 10
failed = [0] * 10
for i in range(test_size): # test_size
    ans = test_labels[i]
    pred = getNumber(test_images[i])
#     print("answer is", ans, "; prediction is", pred)
    if (ans == pred):
        ok[ans] += 1
    else:
        failed[ans] += 1    
print("RESULT")
for i in range(10):
    print("NUMBER =", i, "; OK =", ok[i], "; FAILED =", failed[i])
print("SUM OK =", sum(ok), "; FAILED =", sum(failed))

# 2. SUM OK = 9421 ; FAILED = 579, the worst digits: 7 (112 failed), 8 (106 failed)
# 3. SUM OK = 9484 ; FAILED = 516, the worисst digits: 7 (91 failed), 8 (91 failed)
# 5. SUM OK = 9570 ; FAILED = 430, the worst digits: 7 (79 failed), 8 (69 failed)

RESULT
NUMBER = 0 ; OK = 970 ; FAILED = 10
NUMBER = 1 ; OK = 1122 ; FAILED = 13
NUMBER = 2 ; OK = 982 ; FAILED = 50
NUMBER = 3 ; OK = 971 ; FAILED = 39
NUMBER = 4 ; OK = 952 ; FAILED = 30
NUMBER = 5 ; OK = 836 ; FAILED = 56
NUMBER = 6 ; OK = 920 ; FAILED = 38
NUMBER = 7 ; OK = 949 ; FAILED = 79
NUMBER = 8 ; OK = 905 ; FAILED = 69
NUMBER = 9 ; OK = 963 ; FAILED = 46
SUM OK = 9570 ; FAILED = 430


In [162]:
root = Tk()
root.title("Draw the digit")
root.resizable(0,0)
down = False
def press(event):
    global down
    down = True

def release(event):
    global down
    down = False

def motion(event):
    if down:
        c.create_oval(event.x, event.y, event.x+1, event.y+1, width=8, fill="black")

c = Canvas(root, bg="white", width=400, height= 400)
c.configure(cursor="crosshair")
c.pack()

c.bind("<Button-1>", press)
c.bind("<ButtonRelease-1>", release)

c.bind('<Motion>', motion)

def btn_callback():
    global c
    c.postscript(file="digit.eps")
    img = Image.open("digit.eps")
    img.save("digit.png", "png")
    img = im.convert('1')      # Convert to black&white
    size = 28, 28
    img.thumbnail(size, Image.ANTIALIAS)
    arr = np.array(im)
    img.save("small_digit.png", "png")
    resh = arr.reshape((28*28, 1))
    print(resh.shape)


b = Button(root, text="OK", command=btn_callback)
b.pack()

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Users\Daria\Documents\Anaconda\lib\tkinter\__init__.py", line 1550, in __call__
    return self.func(*args)
  File "<ipython-input-162-f2970b35a634>", line 30, in btn_callback
    img.save("digit.png", "png")
  File "D:\Users\Daria\Documents\Anaconda\lib\site-packages\PIL\Image.py", line 1658, in save
    self.load()
  File "D:\Users\Daria\Documents\Anaconda\lib\site-packages\PIL\EpsImagePlugin.py", line 337, in load
    self.im = Ghostscript(self.tile, self.size, self.fp, scale)
  File "D:\Users\Daria\Documents\Anaconda\lib\site-packages\PIL\EpsImagePlugin.py", line 137, in Ghostscript
    raise WindowsError('Unable to locate Ghostscript on paths')
OSError: Unable to locate Ghostscript on paths


In [165]:
img = Image.open("digit.eps")
img.save("digit.png", "png")

OSError: Unable to locate Ghostscript on paths